# ConversationKGMemory

지식 그래프의 힘을 활용하여 정보를 저장하고 불러옵니다.

이를 통해 모델이 서로 다른 개체 간의 관계를 이해하는 데 도움을 주고, 복잡한 연결망과 역사적 맥락을 기반으로 대응하는 능력을 향상시킵니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

In [3]:
llm = ChatOpenAI(temperature=0)

memory = ConversationKGMemory(llm=llm, return_messages=True)

# 1번째 대화
memory.save_context(
    {"input": "이분은 서울 강남에 거주하는 이민호씨입니다."},
    {"output": "아, 이민호씨는 어디에서 일하시나요?"},
)

# 2번째 대화
memory.save_context(
    {"input": "이민호씨는 네이버의 백엔드 개발자입니다."},
    {"output": "네이버에서 근무하시는군요. 반갑습니다."},
)


In [4]:
memory.load_memory_variables({"input": "이민호씨는 누구입니까?"})

{'history': [SystemMessage(content='On 이민호씨: 이민호씨 거주하는 서울 강남에. 이민호씨 is a 백엔드 개발자. 이민호씨 works at 네이버.', additional_kwargs={}, response_metadata={})]}

## Chain 에 메모리 활용하기

`ConversationChain` 에 `ConversationKGMemory` 를 메모리로 지정하여 대화를 나눈 후 memory 를 확인해 보도록 하겠습니다.


In [5]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(temperature=0)

template = """다음은 인간과 도라에몽 사이의 즐거운 대화입니다. 
도라에몽은 친근하고 상상력이 풍부하며, 비밀도구를 활용해 유쾌한 답변을 해줍니다. 
만약 도라에몽이 어떤 질문에 대해 모른다면 솔직하게 모른다고 말합니다. 
도라에몽은 반드시 아래 "관련 정보"에 포함된 내용만 사용하며, 지어내지 않습니다.

관련 정보:

{history}

대화:
사람: {input}
도라에몽:"""

prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

C:\Users\손용훈\AppData\Local\Temp\ipykernel_13600\3821018149.py:23: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_with_kg = ConversationChain(


첫 번째 대화를 시작합니다. 간단한 인물에 대한 정보를 제공해 보겠습니다.


In [10]:
conversation_with_kg.predict(
    input="안녕 도라에몽! 나는 손용훈이라고 해. 내 친구 노진구는 우리 학교에 전학온 친구야. 잘 부탁해!"
)


'안녕 손용훈! 반가워. 노진구 친구라니, 그럼 우리도 친구가 되어야겠네. 어떤 도움이 필요하면 언제든지 말해줘!'

Shirley 라는 사람에 대한 질문을 진행합니다.


In [13]:
# 노진구 에 대한 질문
conversation_with_kg.memory.load_memory_variables({"input": "노진구는 누구야?"})

{'history': 'On 노진구: 노진구 전학온 우리 학교에.'}